# Аналитика с применением ML. Обучение с учителем. Классификация.

## Вам необходимо построить модель-классификатор по определению диабета.

<div style="text-align: center;\">
    <img src="https://atlas.ru/blog/content/images/size/w2000/2020/11/2020-11-17-Diabetes-1.png" width="700" height="400">
</div>

In [2]:
# Импортируем нужные библиотеки
import pandas as pd # для работы с данными
import numpy as np # для вычислений
import matplotlib.pyplot as plt # для визулизаций
import seaborn as sns # для визулизаций

from sklearn.linear_model import LogisticRegression # Линейная модель
from sklearn import preprocessing # предобработка
from sklearn import metrics # метрики
from sklearn.model_selection import train_test_split # выборка
%matplotlib inline

Повторим, как организуется процесс разработки DS-проектов согласно методологии CRISP-DM.

Этапы модели CRISP-DM:
1. Анализ требований
2. Иследование данных
3. Подготовка данных
4. Моделирование
5. Оценка модели
6. Внедрение

### Анализ требований

**Нам необходимо построить модель-классификатор по определению диабета.** У вас нет никаких ограничений по инструментам, новым полям и способу энкодинга данных. Есть данные для train и для test, нужно предсказать диабет для данных test. 

**Цель: F1-score был больше 0,85**

**Знакомство с данными**

In [9]:
# Считываем данные из таблицы
df = pd.read_csv(r'data\training_data.csv')
df.drop('Unnamed: 0', axis=1, inplace=True) # удаляем ошибочную переменную "Unnamed: 0"
df.set_index("ID", inplace=True) # устанавливаем колонку "ID" как индекс
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
ID,,,,,,,,,
78830894-cdd8-43d1-9655-03db74141b7a,Female,80.0,0,1,never,25.19,6.6,140,0
32262c0e-903a-46b8-9061-b1a2f5a3e9b2,Female,54.0,0,0,No Info,27.32,6.6,80,0
c4868b73-ca90-48ed-af14-3d1f78a5f030,Male,28.0,0,0,never,27.32,5.7,158,0
bec2ef91-5aff-48df-ac16-cb210b5f29fa,Female,36.0,0,0,current,23.45,5.0,155,0
1a6852a8-ee80-4d93-bea0-f0cdd941dc3d,Male,76.0,1,1,current,20.14,4.8,155,0


На каждой строке описано медицинские данные пациента:
1. gender - Пол
2. age - Возраст
3. hypertension - Гипертония
4. heart_disease - Болезни сердца
5. smoking_history - История курения
6. bmi - Индекс массы тела
7. HbA1c_level - Уровень HbA1c
8. blood_glucose_level - Уровень глюкозы в крови
9. diabetes - Диабет (**Целевая переменная**)

In [10]:
df.shape

(73718, 9)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73718 entries, 78830894-cdd8-43d1-9655-03db74141b7a to a98d816c-f44e-4329-930b-58466aaa0bc7
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               73718 non-null  object 
 1   age                  73718 non-null  float64
 2   hypertension         73718 non-null  int64  
 3   heart_disease        73718 non-null  int64  
 4   smoking_history      73718 non-null  object 
 5   bmi                  73718 non-null  float64
 6   HbA1c_level          73718 non-null  float64
 7   blood_glucose_level  73718 non-null  int64  
 8   diabetes             73718 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 5.6+ MB
